## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SSSSBBBB'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
503
+------------------+-------------------+-------------------+------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+-------------------+------------------+-----------+-----------+------+--------------------+------+
|           coeff_1|            coeff_2|            coeff_3|           coeff_4|elevation|   label|latitude|longitude|measurement|              res_1|              res_2|              res_3|              res_4|          res_mean|    station|  total_var|undefs|              vector|  year|
+------------------+-------------------+-------------------+------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+-------------------+------------------+-----------+-----------+------+--------------------+------+
|-4673.181412303238|-2309.3414111954503|  143.7924205220731|-4

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'coeff_4', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_4', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [28]:
#extract longitude and latitude for each station
feature='coeff_2'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, ' + feature
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_2 FROM weather


station    year      coeff_2
0  USC00299820  1984.0 -2309.341411
1  USC00299820  1985.0  -831.394998
2  USC00298011  1968.0  2425.790641
3  USC00299820  1979.0 -1382.988673
4  USC00298011  1960.0  -699.628566

In [29]:
year_station_table=pdf.pivot(index='year', columns='station', values=feature)
year_station_table.head(10)

station  US1NMBR0034  USC00290225  USC00290743  USC00290903  USC00291080  \
year                                                                       
1948.0           NaN          NaN  -147.348495          NaN          NaN   
1949.0           NaN          NaN  -256.331449          NaN          NaN   
1950.0           NaN          NaN          NaN          NaN          NaN   
1951.0           NaN          NaN   -26.494831          NaN          NaN   
1952.0           NaN          NaN     7.046026          NaN          NaN   
1953.0           NaN          NaN          NaN          NaN          NaN   
1954.0           NaN          NaN          NaN          NaN          NaN   
1955.0           NaN          NaN          NaN          NaN          NaN   
1956.0           NaN          NaN    -8.994599          NaN     6.133287   
1957.0           NaN          NaN          NaN          NaN          NaN   

station  USC00291086  USC00291647  USC00291982  USC00292100  USC00292139  \
year                                                                       
1948.0     92.414234          NaN          NaN          NaN          NaN   
1949.0           NaN          NaN          NaN          NaN          NaN   
1950.0           NaN          NaN          NaN          NaN          NaN   
1951.0           NaN          NaN          NaN          NaN          NaN   
1952.0           NaN          NaN          NaN          NaN          NaN   
1953.0           NaN    82.543097          NaN          NaN          NaN   
1954.0           NaN          NaN          NaN          NaN          NaN   
1955.0           NaN          NaN          NaN          NaN   -72.462037   
1956.0           NaN          NaN          NaN          NaN  -282.876221   
1957.0           NaN          NaN          NaN          NaN          NaN   

station     ...       USC00298830  USC00298834  USC00299031  USC00299113  \
year        ...                                                            
1948.0      ...               NaN          NaN          NaN   -16.032611   
1949.0      ...               NaN          NaN          NaN          NaN   
1950.0      ...               NaN          NaN          NaN          NaN   
1951.0      ...               NaN          NaN          NaN          NaN   
1952.0      ...               NaN          NaN          NaN          NaN   
1953.0      ...               NaN          NaN          NaN          NaN   
1954.0      ...               NaN          NaN          NaN          NaN   
1955.0      ...               NaN          NaN          NaN          NaN   
1956.0      ...        -50.350314          NaN          NaN          NaN   
1957.0      ...               NaN          NaN          NaN          NaN   

station  USC00299193  USC00299496  USC00299820  USW00023049  USW00023054  \
year                                                                       
1948.0           NaN          NaN          NaN          NaN    77.553174   
1949.0           NaN          NaN          NaN   -31.578275    -9.399484   
1950.0           NaN          NaN          NaN          NaN          NaN   
1951.0           NaN          NaN          NaN          NaN          NaN   
1952.0           NaN          NaN          NaN          NaN    39.637198   
1953.0           NaN          NaN  -271.133493          NaN    41.333440   
1954.0           NaN          NaN          NaN          NaN   -32.798985   
1955.0           NaN          NaN  -356.979997          NaN          NaN   
1956.0     20.546444          NaN  -554.706544          NaN          NaN   
1957.0           NaN          NaN   -42.953996          NaN          NaN   

station  USW00093057  
year                  
1948.0           NaN  
1949.0           NaN  
1950.0           NaN  
1951.0           NaN  
1952.0           NaN  
1953.0           NaN  
1954.0           NaN  
1955.0           NaN  
1956.0           NaN  
1957.0           NaN  

[10 rows x 52 columns]

### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the MS before and after subtracting either  the row or the column vector.

In [30]:
def MS(Mat):
    return np.nanmean(Mat**2)

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total MS                   = ',MS(year_station_table)
print 'MS removing mean-by-station= ',MS(tbl_minus_station)
print 'MS removing mean-by-year   = ',MS(tbl_minus_year)
print 'fraction explained station =', float(MS(year_station_table)-MS(tbl_minus_station))*100/float(MS(year_station_table))
print 'fraction explained year =', float(MS(year_station_table)-MS(tbl_minus_year))*100/float(MS(year_station_table))

total MS                   =  318159.534163
MS removing mean-by-station=  264675.69173
MS removing mean-by-year   =  211595.745315
fraction explained station = 16.8103849454
fraction explained year = 33.4938222514


In [9]:
T=year_station_table
print 'initial MS=',MS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',MS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',MS(T)

initial MS= 2224620.20611
0 after removing mean by year    = 1802372.49668
0 after removing mean by stations= 716839.488468
1 after removing mean by year    = 583975.104811
1 after removing mean by stations= 557413.685991
2 after removing mean by year    = 550052.583665
2 after removing mean by stations= 547598.335719
3 after removing mean by year    = 546691.913307
3 after removing mean by stations= 546335.688662
4 after removing mean by year    = 546189.619218
4 after removing mean by stations= 546127.752043
